# Attribute Table Data Extraction Codes

### Setting up the files

In [ ]:
import os
import arcpy
import pandas as pd
from arcpy.sa import *
from datetime import datetime

In [ ]:
gdb_path = r"D:\My_project\MXD\Trial_for_coding\Datatset\Meter_trial_4.gdb"
arcpy.env.overwriteOutput = True
arcpy.env.workspace = gdb_path

In [ ]:
mclass = []
fclass = arcpy.ListFeatureClasses()
for fc in fclass:
    file_name = os.path.basename(fc)
    if file_name.lower().startswith("shade"):
            
        # Describe the feature class to get its properties
        desc = arcpy.Describe(fc)
        info = desc.shapeType
        #print(info)
        # Check if the feature class is a multipatch
        if info == "MultiPatch":
            mclass.append(fc)

In [ ]:
not_avail = []
f_list = []
for i in range(1, 217):
    names_to_check = [
        "Shade_Building_{}_2m_v2_panels".format(i),
        "Shade_Building_{}_5m_v2_panels".format(i),
        "Shade_Building_{}_10m_v2_panels".format(i),
        "Shade_Building_{}_30m_v2_panels".format(i),
        "Shade_Tree_{}_2m_v2_panels".format(i),
        "Shade_Tree_{}_5m_v2_panels".format(i),
        "Shade_Tree_{}_10m_v2_panels".format(i),
        "Shade_Tree_{}_30m_v2_panels".format(i),
        "Shade_Combined_{}_2m_v2_panels".format(i),
        "Shade_Combined_{}_5m_v2_panels".format(i),
        "Shade_Combined_{}_10m_v2_panels".format(i),
        "Shade_Combined_{}_30m_v2_panels".format(i)
    ]
    
    for name in names_to_check:
        if name not in mclass:
            not_avail.append(name)
        else:
            f_list.append(name)
            

In [ ]:
print(len(f_list))

### Delete the field if exits

In [ ]:
field_name = 'Where_pane_lies'
total = (len(f_list))
i = 1
   
for file in f_list:
    if field_name in [f.name for f in arcpy.ListFields(file)]:
        arcpy.DeleteField_management(file, field_name)
    percent = (i*100)/(total)
    print(f"{percent:.2f}")
    i = i + 1

54.44
54.48
54.51
54.55
54.59
54.63
54.67
54.71
54.75
54.78
54.82
54.86
54.90
54.94
54.98
55.02
55.05
55.09
55.13
55.17
55.21
55.25
55.29
55.32
55.36
55.40
55.44
55.48
55.52
55.56
55.59
55.63
55.67
55.71
55.75
55.79
55.83
55.86
55.90
55.94
55.98
56.02
56.06
56.10
56.13
56.17
56.21
56.25
56.29
56.33
56.37
56.40
56.44
56.48
56.52
56.56
56.60
56.64
56.67
56.71
56.75
56.79
56.83
56.87
56.91
56.94
56.98
57.02
57.06
57.10
57.14
57.18
57.21
57.25
57.29
57.33
57.37
57.41
57.45
57.48
57.52
57.56
57.60
57.64
57.68
57.72
57.75
57.79
57.83
57.87
57.91
57.95
57.99
58.02
58.06
58.10
58.14
58.18
58.22
58.26
58.29
58.33
58.37
58.41
58.45
58.49
58.53
58.56
58.60
58.64
58.68
58.72
58.76
58.80
58.83
58.87
58.91
58.95
58.99
59.03
59.07
59.10
59.14
59.18
59.22
59.26
59.30
59.34
59.38
59.41
59.45
59.49
59.53
59.57
59.61
59.65
59.68
59.72
59.76
59.80
59.84
59.88
59.92
59.95
59.99
60.03
60.07
60.11
60.15
60.19
60.22
60.26
60.30
60.34
60.38
60.42
60.46
60.49
60.53
60.57
60.61
60.65
60.69
60.73
60.76
60.80
60.8

# Attribute Table Field Addition code

### Attribute Filed (where Pane lies) Addition

In [ ]:
import re
import glob
from datetime import datetime

footprint =r'D:\My_project\MXD\Raster_Files\Raster_Files.gdb\All_Building_Clipped_Buffer_20'
gdb_path = r"D:\My_project\MXD\Trial_for_coding\Datatset\Meter_trial_4.gdb"
bldg = r"D:\My_project\MXD\Trial_for_coding\Datatset\Meter_trial.gdb\Final_Building_v7_2_m"

field_name = 'Where_pane_lies'
field_alias = 'Panetype'

i = 1

TSTART = datetime.now()

for file in f_list:
 
    # Add the field if it does not exist
    if field_name not in [f.name for f in arcpy.ListFields(file)]:

        arcpy.management.AddField(file, field_name, 'TEXT', field_alias = field_alias, field_is_nullable='NULLABLE', 
                                  field_is_required='NON_REQUIRED')
    
    layer_name = "temp_layer"    
    # Create a feature layer from the feature class
    arcpy.management.MakeFeatureLayer(file, layer_name)

    # Select features that have their center in the footprint
    arcpy.management.SelectLayerByLocation(layer_name, "HAVE_THEIR_CENTER_IN", footprint, "", "NEW_SELECTION")

    arcpy.management.SelectLayerByLocation(layer_name, "HAVE_THEIR_CENTER_IN", bldg, "", "ADD_TO_SELECTION")

    # Update the selected features to 'building'
    with arcpy.da.UpdateCursor(layer_name, [field_name]) as cursor:
        for row in cursor:
            row[0] = 'building'
            cursor.updateRow(row)

    # Remove the selection
    arcpy.management.SelectLayerByAttribute(layer_name, "SWITCH_SELECTION")

    # Update the selected features to 'ground'
    with arcpy.da.UpdateCursor(layer_name, [field_name]) as cursor:
        for row in cursor:
            row[0] = 'ground'
            cursor.updateRow(row)

    # Remove the selection
    arcpy.management.SelectLayerByAttribute(layer_name, "CLEAR_SELECTION")

    # Delete the temporary feature layer
    arcpy.management.Delete(layer_name)
    
    percent = (i*100)/(total)
    print(f"{percent:.2f}")

    i = i + 1

TEND = datetime.now()
print(f'Processing time: {TEND-TSTART} [HH:MM:SS]')

54.40
54.44
54.48
54.51
54.55
54.59
54.63
54.67
54.71
54.75
54.78
54.82
54.86
54.90
54.94
54.98
55.02
55.05
55.09
55.13
55.17
55.21
55.25
55.29
55.32
55.36
55.40
55.44
55.48
55.52
55.56
55.59
55.63
55.67
55.71
55.75
55.79
55.83
55.86
55.90
55.94
55.98
56.02
56.06
56.10
56.13
56.17
56.21
56.25
56.29
56.33
56.37
56.40
56.44
56.48
56.52
56.56
56.60
56.64
56.67
56.71
56.75
56.79
56.83
56.87
56.91
56.94
56.98
57.02
57.06
57.10
57.14
57.18
57.21
57.25
57.29
57.33
57.37
57.41
57.45
57.48
57.52
57.56
57.60
57.64
57.68
57.72
57.75
57.79
57.83
57.87
57.91
57.95
57.99
58.02
58.06
58.10
58.14
58.18
58.22
58.26
58.29
58.33
58.37
58.41
58.45
58.49
58.53
58.56
58.60
58.64
58.68
58.72
58.76
58.80
58.83
58.87
58.91
58.95
58.99
59.03
59.07
59.10
59.14
59.18
59.22
59.26
59.30
59.34
59.38
59.41
59.45
59.49
59.53
59.57
59.61
59.65
59.68
59.72
59.76
59.80
59.84
59.88
59.92
59.95
59.99
60.03
60.07
60.11
60.15
60.19
60.22
60.26
60.30
60.34
60.38
60.42
60.46
60.49
60.53
60.57
60.61
60.65
60.69
60.73
60.76
60.8

### Attribute Table Export as CSV

In [ ]:
at_table = []
fields = []
aliases = []

out_path = r"D:\My_project\Datasets\Excel\Data_5_0"
gdb_path = r"D:\My_project\MXD\Trial_for_coding\Datatset\Meter_trial_4.gdb"

arcpy.env.overwriteOutput = True
arcpy.env.workspace = gdb_path

total = (len(f_list))
i = 1

for pane in f_list:
    
    fname = os.path.basename(pane)
    
    for field in arcpy.ListFields(fname):
        fields.append(field.name)
        aliases.append(field.aliasName)

    with arcpy.da.SearchCursor(fname, fields) as cursor:
        for row in cursor:
            at_table.append(row)
    df = pd.DataFrame(at_table, columns=aliases)
    
    os.chdir(out_path)
    #os.chdir(t_out)
    df.to_csv('{}.csv'.format(fname))
    
    df = pd.DataFrame()
    
    at_table = []
    fields = []
    aliases = []
    
    os.chdir(gdb_path)
    
    t_percent = (i*100)/total
    
    print (f"{t_percent:.2f}")
    i = i + 1

54.40
54.44
54.48
54.51
54.55
54.59
54.63
54.67
54.71
54.75
54.78
54.82
54.86
54.90
54.94
54.98
55.02
55.05
55.09
55.13
55.17
55.21
55.25
55.29
55.32
55.36
55.40
55.44
55.48
55.52
55.56
55.59
55.63
55.67
55.71
55.75
55.79
55.83
55.86
55.90
55.94
55.98
56.02
56.06
56.10
56.13
56.17
56.21
56.25
56.29
56.33
56.37
56.40
56.44
56.48
56.52
56.56
56.60
56.64
56.67
56.71
56.75
56.79
56.83
56.87
56.91
56.94
56.98
57.02
57.06
57.10
57.14
57.18
57.21
57.25
57.29
57.33
57.37
57.41
57.45
57.48
57.52
57.56
57.60
57.64
57.68
57.72
57.75
57.79
57.83
57.87
57.91
57.95
57.99
58.02
58.06
58.10
58.14
58.18
58.22
58.26
58.29
58.33
58.37
58.41
58.45
58.49
58.53
58.56
58.60
58.64
58.68
58.72
58.76
58.80
58.83
58.87
58.91
58.95
58.99
59.03
59.07
59.10
59.14
59.18
59.22
59.26
59.30
59.34
59.38
59.41
59.45
59.49
59.53
59.57
59.61
59.65
59.68
59.72
59.76
59.80
59.84
59.88
59.92
59.95
59.99
60.03
60.07
60.11
60.15
60.19
60.22
60.26
60.30
60.34
60.38
60.42
60.46
60.49
60.53
60.57
60.61
60.65
60.69
60.73
60.76
60.8

# Data Summarization Code Below

In [ ]:
import os
import re
import glob
import arcpy
import pandas as pd
from arcpy.sa import *
from datetime import datetime

In [ ]:
gdb_path = r"D:\My_project\MXD\Trial_for_coding\Datatset\Shadow_Analysis.gdb"

in_path = r"D:\My_project\Datasets\Excel\Data_5_0"
out_path = r"D:\My_project\Datasets\Excel"

os.chdir(in_path)

arcpy.env.overwriteOutput = True
arcpy.env.workspace = gdb_path


In [ ]:
# Get the list of all CSV files in the directory
csv_file = glob.glob(os.path.join(in_path, "*.csv"))

# Extract the base filenames from the full paths
csv_files = [os.path.basename(file) for file in csv_file]
print(len(csv_files))

In [ ]:
TSTART = datetime.now()

total = len(csv_file)

shadow_h_values = [0, 0.5, 1, 1.5, 2, 2.5]

panel_type = ['building','ground']

w_or_type = [['east','west','north', 'south'], ['up']]

cols = ["Subarea", "Shade_Type", "Orientation_Type","Resolution", "Area_Total", "Or_Type_Tot_Area", "Area_0", "Area_0.5", 
        "Area_1", "Area_1.5", "Area_2", "Area_2.5", "Panel_Count"]

summary_df = pd.DataFrame(columns = cols)

i = 1

for file in csv_files:
    
    pattern = re.compile(r'Shade_(Combined|Tree|Building)_(\d+)_(2|5|10|30)m_v2_panels\.csv')
    
    match = pattern.match(file)

    if match:
        category = match.group(1)  # Combined, Tree, or Buildings
        number = match.group(2)  # The extracted number'
        resolution = match.group(3)  # The extracted resolution (2, 5, 10, or 30)

    df = pd.read_csv(file)
    
    df = df.dropna(subset=['Area'])
    
    df['Area'] = df['Area'].astype(float)
    
    tot_area = df['Area'].sum()
    
    
    for ptype in panel_type:
        
        df_building = df[df['Panetype'] ==  ptype]
        
        if ptype == 'building':
            
            for or_type in w_or_type:
        
                df_filtered = df_building[df_building['World_Orientation'].isin(or_type)]
            
                if or_type == ['east','west','north', 'south']:
            
                    Orientation_Type = 'Facade'
                    
                    tot_area_ortype = df_filtered['Area'].sum()
                    
                    p_count= len(df_filtered)
                
                else:                  
                    
                    Orientation_Type = 'Roof'
                    
                    p_count = len(df_filtered)
                    
                    tot_area_ortype = df_filtered['Area'].sum()
                    
                        
                area_0 = df_filtered[df_filtered['shadow_h'] == 0]['Area'].sum()

                area_0_5 = df_filtered[df_filtered['shadow_h'] == 0.5]['Area'].sum()

                area_1= df_filtered[df_filtered['shadow_h'] == 1]['Area'].sum()

                area_1_5 = df_filtered[df_filtered['shadow_h'] == 1.5]['Area'].sum()

                area_2 = df_filtered[df_filtered['shadow_h'] == 2]['Area'].sum()

                area_2_5 = df_filtered[df_filtered['shadow_h'] == 2.5]['Area'].sum()  
                   
                new_row = pd.DataFrame([{
                        'Subarea': number,
                        'Shade_Type': category,
                        'Orientation_Type': Orientation_Type,
                        'Resolution': resolution,
                        'Area_Total': tot_area,
                        'Or_Type_Tot_Area': tot_area_ortype,
                        'Area_0': area_0,
                        'Area_0.5': area_0_5,
                        'Area_1': area_1,
                        'Area_1.5': area_1_5,
                        'Area_2': area_2,
                        'Area_2.5': area_2_5,
                        'Panel_Count': p_count
                    }])

                summary_df = pd.concat([summary_df, new_row], ignore_index=True)            
            
        else:

            Orientation_Type = 'Ground'

            df_filtered = df[df['Panetype'] ==  ptype]

            tot_area_ortype = df_filtered['Area'].sum()
            
            p_count = len(df_filtered)

            area_0 = df_filtered[df_filtered['shadow_h'] == 0]['Area'].sum()

            area_0_5 = df_filtered[df_filtered['shadow_h'] == 0.5]['Area'].sum()

            area_1= df_filtered[df_filtered['shadow_h'] == 1]['Area'].sum()

            area_1_5 = df_filtered[df_filtered['shadow_h'] == 1.5]['Area'].sum()

            area_2 = df_filtered[df_filtered['shadow_h'] == 2]['Area'].sum()

            area_2_5 = df_filtered[df_filtered['shadow_h'] == 2.5]['Area'].sum()  


            new_row = pd.DataFrame([{
                    'Subarea': number,
                    'Shade_Type': category,
                    'Orientation_Type': Orientation_Type,
                    'Resolution': resolution,
                    'Area_Total': tot_area,
                    'Or_Type_Tot_Area': tot_area_ortype,
                    'Area_0': area_0,
                    'Area_0.5': area_0_5,
                    'Area_1': area_1,
                    'Area_1.5': area_1_5,
                    'Area_2': area_2,
                    'Area_2.5': area_2_5,
                    'Panel_Count': p_count
                }])

            summary_df = pd.concat([summary_df, new_row], ignore_index=True)
               
    print(f'Total Number of files processed: {i}')
        
    i = i + 1
        
TEND = datetime.now()

print(f'Processing time: {TEND-TSTART} [HH:MM:SS]')

Total Number of files processed: 226
Total Number of files processed: 227
Total Number of files processed: 228
Total Number of files processed: 229
Total Number of files processed: 230
Total Number of files processed: 231
Total Number of files processed: 232
Total Number of files processed: 233
Total Number of files processed: 234
Total Number of files processed: 235
Total Number of files processed: 236
Total Number of files processed: 237
Total Number of files processed: 238
Total Number of files processed: 239
Total Number of files processed: 240
Total Number of files processed: 241
Total Number of files processed: 242
Total Number of files processed: 243
Total Number of files processed: 244
Total Number of files processed: 245
Total Number of files processed: 246
Total Number of files processed: 247
Total Number of files processed: 248
Total Number of files processed: 249
Total Number of files processed: 250
Total Number of files processed: 251
Total Number of files processed: 252
T

Total Number of files processed: 450
Total Number of files processed: 451
Total Number of files processed: 452
Total Number of files processed: 453
Total Number of files processed: 454
Total Number of files processed: 455
Total Number of files processed: 456
Total Number of files processed: 457
Total Number of files processed: 458
Total Number of files processed: 459
Total Number of files processed: 460
Total Number of files processed: 461
Total Number of files processed: 462
Total Number of files processed: 463
Total Number of files processed: 464
Total Number of files processed: 465
Total Number of files processed: 466
Total Number of files processed: 467
Total Number of files processed: 468
Total Number of files processed: 469
Total Number of files processed: 470
Total Number of files processed: 471
Total Number of files processed: 472
Total Number of files processed: 473
Total Number of files processed: 474
Total Number of files processed: 475
Total Number of files processed: 476
T

Total Number of files processed: 674
Total Number of files processed: 675
Total Number of files processed: 676
Total Number of files processed: 677
Total Number of files processed: 678
Total Number of files processed: 679
Total Number of files processed: 680
Total Number of files processed: 681
Total Number of files processed: 682
Total Number of files processed: 683
Total Number of files processed: 684
Total Number of files processed: 685
Total Number of files processed: 686
Total Number of files processed: 687
Total Number of files processed: 688
Total Number of files processed: 689
Total Number of files processed: 690
Total Number of files processed: 691
Total Number of files processed: 692
Total Number of files processed: 693
Total Number of files processed: 694
Total Number of files processed: 695
Total Number of files processed: 696
Total Number of files processed: 697
Total Number of files processed: 698
Total Number of files processed: 699
Total Number of files processed: 700
T

Total Number of files processed: 898
Total Number of files processed: 899
Total Number of files processed: 900
Total Number of files processed: 901
Total Number of files processed: 902
Total Number of files processed: 903
Total Number of files processed: 904
Total Number of files processed: 905
Total Number of files processed: 906
Total Number of files processed: 907
Total Number of files processed: 908
Total Number of files processed: 909
Total Number of files processed: 910
Total Number of files processed: 911
Total Number of files processed: 912
Total Number of files processed: 913
Total Number of files processed: 914
Total Number of files processed: 915
Total Number of files processed: 916
Total Number of files processed: 917
Total Number of files processed: 918
Total Number of files processed: 919
Total Number of files processed: 920
Total Number of files processed: 921
Total Number of files processed: 922
Total Number of files processed: 923
Total Number of files processed: 924
T

Total Number of files processed: 1118
Total Number of files processed: 1119
Total Number of files processed: 1120
Total Number of files processed: 1121
Total Number of files processed: 1122
Total Number of files processed: 1123
Total Number of files processed: 1124
Total Number of files processed: 1125
Total Number of files processed: 1126
Total Number of files processed: 1127
Total Number of files processed: 1128
Total Number of files processed: 1129
Total Number of files processed: 1130
Total Number of files processed: 1131
Total Number of files processed: 1132
Total Number of files processed: 1133
Total Number of files processed: 1134
Total Number of files processed: 1135
Total Number of files processed: 1136
Total Number of files processed: 1137
Total Number of files processed: 1138
Total Number of files processed: 1139
Total Number of files processed: 1140
Total Number of files processed: 1141
Total Number of files processed: 1142
Total Number of files processed: 1143
Total Number

...

Total Number of files processed: 1553
Total Number of files processed: 1554
Total Number of files processed: 1555
Total Number of files processed: 1556
Total Number of files processed: 1557
Total Number of files processed: 1558
Total Number of files processed: 1559
Total Number of files processed: 1560
Total Number of files processed: 1561
Total Number of files processed: 1562
Total Number of files processed: 1563
Total Number of files processed: 1564
Total Number of files processed: 1565
Total Number of files processed: 1566
Total Number of files processed: 1567
Total Number of files processed: 1568
Total Number of files processed: 1569
Total Number of files processed: 1570
Total Number of files processed: 1571
Total Number of files processed: 1572
Total Number of files processed: 1573
Total Number of files processed: 1574
Total Number of files processed: 1575
Total Number of files processed: 1576
Total Number of files processed: 1577
Total Number of files processed: 1578
Total Number

Total Number of files processed: 1770
Total Number of files processed: 1771
Total Number of files processed: 1772
Total Number of files processed: 1773
Total Number of files processed: 1774
Total Number of files processed: 1775
Total Number of files processed: 1776
Total Number of files processed: 1777
Total Number of files processed: 1778
Total Number of files processed: 1779
Total Number of files processed: 1780
Total Number of files processed: 1781
Total Number of files processed: 1782
Total Number of files processed: 1783
Total Number of files processed: 1784
Total Number of files processed: 1785
Total Number of files processed: 1786
Total Number of files processed: 1787
Total Number of files processed: 1788
Total Number of files processed: 1789
Total Number of files processed: 1790
Total Number of files processed: 1791
Total Number of files processed: 1792
Total Number of files processed: 1793
Total Number of files processed: 1794
Total Number of files processed: 1795
Total Number

Total Number of files processed: 1986
Total Number of files processed: 1987
Total Number of files processed: 1988
Total Number of files processed: 1989
Total Number of files processed: 1990
Total Number of files processed: 1991
Total Number of files processed: 1992
Total Number of files processed: 1993
Total Number of files processed: 1994
Total Number of files processed: 1995
Total Number of files processed: 1996
Total Number of files processed: 1997
Total Number of files processed: 1998
Total Number of files processed: 1999
Total Number of files processed: 2000
Total Number of files processed: 2001
Total Number of files processed: 2002
Total Number of files processed: 2003
Total Number of files processed: 2004
Total Number of files processed: 2005
Total Number of files processed: 2006
Total Number of files processed: 2007
Total Number of files processed: 2008
Total Number of files processed: 2009
Total Number of files processed: 2010
Total Number of files processed: 2011
Total Number

Total Number of files processed: 2203
Total Number of files processed: 2204
Total Number of files processed: 2205
Total Number of files processed: 2206
Total Number of files processed: 2207
Total Number of files processed: 2208
Total Number of files processed: 2209
Total Number of files processed: 2210
Total Number of files processed: 2211
Total Number of files processed: 2212
Total Number of files processed: 2213
Total Number of files processed: 2214
Total Number of files processed: 2215
Total Number of files processed: 2216
Total Number of files processed: 2217
Total Number of files processed: 2218
Total Number of files processed: 2219
Total Number of files processed: 2220
Total Number of files processed: 2221
Total Number of files processed: 2222
Total Number of files processed: 2223
Total Number of files processed: 2224
Total Number of files processed: 2225
Total Number of files processed: 2226
Total Number of files processed: 2227
Total Number of files processed: 2228
Total Number

Total Number of files processed: 2420
Total Number of files processed: 2421
Total Number of files processed: 2422
Total Number of files processed: 2423
Total Number of files processed: 2424
Total Number of files processed: 2425
Total Number of files processed: 2426
Total Number of files processed: 2427
Total Number of files processed: 2428
Total Number of files processed: 2429
Total Number of files processed: 2430
Total Number of files processed: 2431
Total Number of files processed: 2432
Total Number of files processed: 2433
Total Number of files processed: 2434
Total Number of files processed: 2435
Total Number of files processed: 2436
Total Number of files processed: 2437
Total Number of files processed: 2438
Total Number of files processed: 2439
Total Number of files processed: 2440
Total Number of files processed: 2441
Total Number of files processed: 2442
Total Number of files processed: 2443
Total Number of files processed: 2444
Total Number of files processed: 2445
Total Number

In [ ]:
os.chdir(out_path)
summary_df.to_csv('Shade_Summary_Table_5_0.csv')